# Автоматизация ответов на часто задаваемые вопросы в навыке для "Алисы" с помощью библиотеки DeepPavlov

В этом туториале я расскажу как разработать и подключить навык классификации текста на основе библиотеки **DeepPavlov** к **Яндекс.Алиса**. Наш навык сможет приветствовать, прощаться и отвечать на вопросы (на основе списка FAQ - часто задаваемых вопросов). Более детальное руководство по классификации текста на основе библиотеки DeepPavlov вы сможете найти в статье [Simple intent recognition and question answering with DeepPavlov](https://medium.com/deeppavlov).

## Установка

Для начала установите Python 3.6 и активируйте среду разработки. Затем скачайте библиотеку DeepPavlov.

In [ ]:
!source activate py36
!pip install -q deeppavlov

## Разработка

Создайте скиллы для приветствия, прощания, и скилл на случай если запрос пользователя не удовлетворяет ни одной из категорий. Параметр `responses` определяет варианты ответа навыка на запросы из параметра `patters`. Если при определении навыка параметр `pattens` не задан, то этот навык становится навыком "заглушкой", то есть он вызывается в том случае если ни один из навыков не сработал. Параметр `default_confidence` задает минимальный порог уверенности при котором навык-заглушка сработает.

In [ ]:
from deeppavlov.skills.pattern_matching_skill import PatternMatchingSkill

hello = PatternMatchingSkill(responses=['Привет', 'Приветствую'], patterns=['Привет', 'Здравствуйте'])
bye = PatternMatchingSkill(responses=['Пока', 'Всего доброго'], patterns=['Пока', 'До свидания'])
fallback = PatternMatchingSkill(responses=['Пожалуйста перефразируйте'], default_confidence = 0.3)

Создайте навык класса `SimilarityMatchingSkill`, который отвечает на запрос пользователя на основе списка часто задаваемых вопросов. Объект имеет следующие параметры
* `data_path` - путь к csv файлу с данными
* `x_col_name` - имя колонки с вопросами в csv файле (Question, по умолчанию)
* `y_col_name` - имя колонки с ответами в csv файле (Answer, по умолчанию)
* `edit_dict` - `dict` с параметрами конфигурации для перезаписи
* `save_load_path` - путь куда сохранить натренированную модель
* `train` - тренировать ли модель?

In [ ]:
from deeppavlov.contrib.skills.similarity_matching_skill import SimilarityMatchingSkill

faq = SimilarityMatchingSkill(data_path = 'http://files.deeppavlov.ai/faq/dataset_ru.csv',
                              x_col_name = 'Question', 
                              y_col_name = 'Answer',
                              save_load_path = './model',
                              config_type = 'tfidf_autofaq',
                              train = True)

`SimilarityMatchingSkill` это класс навыка, который позволяет классифицировать вопросы пользователя. Натренировав модель и сохранив в `save_load_path`, вы можете использовать ее указав путь загрузки `faq = SimilarityMatchingSkill(load_path='./model')`.

Последний шаг объединить все навыки в агент, и настроить параметр выбора навыка. Параметр `HighestConfidenceSelector` определяет, что будет вызван навык с наивысшей уверенностью (`confidence`).

In [ ]:
from deeppavlov.agents.default_agent.default_agent import DefaultAgent
from deeppavlov.agents.processors.highest_confidence_selector import HighestConfidenceSelector

agent = DefaultAgent([hello, bye, faq, fallback], skills_selector=HighestConfidenceSelector())

Убедитесь, что агент корректо отвечает на запросы.

In [ ]:
print(agent(['Привет', 'где будет школа?', 'как получить задание?', 'мне нужна помощь', 'Пока']))

Далее запустите сервер с указанием пути для запросов `endpoint='faq'` и порта подключения `port=5000`

In [ ]:
from deeppavlov.utils.alice import start_agent_server

start_agent_server(agent, host='0.0.0.0', port=5000, endpoint='/faq')

Обратите внимание, что Яндекс.Диалоги в качестве **Webhook URL** требует указывать сервер с внешним IP адресом и доступом по протоколу https. Для быстрого прототипирования вы можете использовать [ngrok](https://ngrok.com/). **Ngrok** позволит вам создавать туннель для доступа к вашему серверу с **DeepPavlov** в локальной сети, для этого запустите `ngrok http 5000` на вашем сервере с DeepPavlov. В ответ на это будет создано два туннеля, по одному на протоколы http и https. Скопируйте адрес туннеля для https, добавьте к линку эндпоинт `/faq`, итоговый линк будет **Webhook URL** для нашего Яндекс.Диалога. Далее заполните поля необходимые для сохранения черновика диалога. Сохраните черновик и перейдите на вкладку Тестирование.

# Ссылки

[DeepPavlov repository](https://github.com/deepmipt/DeepPavlov)

[DeepPavlov demo page](https://demo.ipavlov.ai)

[DeepPavlov documentation](https://docs.deeppavlov.ai)

[DeepPavlov blog](https://medium.com/deeppavlov)

[DeepPavlov forum](https://forum.ipavlov.ai)